### Final project

Each of you has been randomly assigned a list of stocks (tickers). Download 10 years of monthly stock price data for the first 10 stocks, starting January 2010. Only use the first 10 stocks: if there is no data available for a particular stock (or any other problem with the data), replace it with a stock out of the remaining 10.

Your answers should be short and concise, but informative. Try to write robust code, i.e. try to automate as much as possible. Also, at least 80% of your code should be based on what we have learnt in class. The remaining 20% can be additional packages, functions etc.

You will be graded based on:
* Code efficiency and readability;
* Financial knowledge;
* Creative problem solving approach.

### Portfolio hedging using options

##### 1. Calculate risk-free rate without using special packages. A risk-free bond, paying 4% annual coupon, matures in 5 years. It is trading at 1,069.69 USD and the next coupon is paid one year from today. Use this bond's yield as a risk-free rate for further calculations. If you can't calculate the rate, use 2% as the risk-free rate for further calculations.

In [ ]:
import numpy as np

def NPV(cf, r, p):
    cf = np.array(cf)
    return np.dot(cf, (1 / (1 + r) ** (1 / p)) ** np.arange(len(cf)))

def IRR(cf, p=1):
    tolerance = 0.000001
    rate = 0
    npv = NPV(cf, rate, p)
    iteration = 0
    factor = 10
    if npv > tolerance:
        while npv > tolerance or npv < -tolerance:
            while ((-1) ** iteration) * npv > 0:
                rate += ((-1) ** iteration) / (factor ** (iteration + 1))
                npv = NPV(cf, rate, 1)
            iteration += 1
    elif npv < -tolerance:
        while npv > tolerance or npv < -tolerance:
            while ((-1) ** (iteration + 1)) * npv > 0:
                rate += ((-1) ** (iteration + 1)) / (factor ** (iteration + 1))
                npv = NPV(cf, rate, 1)
            iteration += 1
    return (1 + rate) ** p - 1

#Yield of a bond is similar to an IRR of a project
cf = [-1069.69, 40, 40, 40, 40, 1040]
rf = round(IRR(cf), 3)
print('The risk-free rate is {}%.'.format(rf * 100))

##### 2. Using the 10 stocks from before, plot the efficient frontier (no short selling) and draw the capital market line (line connecting risk-free rate and the maximum Sharpe ratio portfolio). Assume you want to invest  100,000 USD in a maximum Sharpe ratio portfolio. How much would you invest in each stock?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sco
import yfinance as yf

data = yf.download('MMC WHR HON SRCL AFL SYY IBM EBAY COF KEY', start = '2010-01-01', end = '2019-12-31', period = '1d')
data = data['Adj Close'].resample('1m').last()

rets = np.log(data / data.shift(1))
rets.describe()

In [ ]:
#To draw the efficient frontier, I use max_ret for the upper bound of the frontier
max_ret = rets.mean().max() * 12
round(max_ret, 3)

In [ ]:
def portfolio_ret(weights):
    return np.dot(rets.mean(), weights) * 12

def portfolio_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(rets.cov(), weights)) * 12)

cons = ({'type': 'eq',
         'fun': lambda x: portfolio_ret(x) - target_ret},
        {'type': 'eq',
         'fun': lambda x: np.sum(x) - 1})  

bnds = tuple((0, 1) for x in range(rets.shape[1]))

#max_ret is the upper bound for target returns
target_rets = np.linspace(0.1, round(max_ret, 3), 50)
target_vols = []

for target_ret in target_rets:
    res = sco.minimize(portfolio_vol,
                       [1 / rets.shape[1]] * rets.shape[1],
                       method='SLSQP',
                       bounds=bnds,
                       constraints=cons)
    target_vols.append(res['fun'])
target_vols = np.array(target_vols)

#Just the efficient frontier
plt.figure(figsize=(10, 6))
plt.grid(True)
plt.plot(target_vols, target_rets,
         'b')
plt.xlabel('Volatility of returns')
plt.ylabel('Expected returns')
plt.title('Efficient frontier using 10 stocks')

In [ ]:
#Obtain the optimal parameters to draw the capital market line
import scipy.interpolate as sci

index = np.argmin(target_vols)
t_vols = target_vols[index:]
t_rets = target_rets[index:]

tck = sci.splrep(t_vols, t_rets)

def f(x):
    return sci.splev(x, tck, der=0)
         
def df(x):
    return sci.splev(x, tck, der=1)

def equations(p, rf=rf):
    equation_1 = rf - p[0]
    equation_2 = rf + p[1] * p[2] - f(p[2])
    equation_3 = p[1] - df(p[2])
    return equation_1, equation_2, equation_3

opt = sco.fsolve(equations, [0.025, 1.2, 0.12])  
print('Optimal parameters are: intercept={}, slope coefficient={} and the x-value at which Sharpe ratio is maximized={}.'\
     .format(round(opt[0], 3), round(opt[1], 3), round(opt[2], 3)))

In [ ]:
#Draw the efficient frontier and the capital market line together
plt.figure(figsize=(10, 6))
plt.plot(t_vols, t_rets,
         'b')
cx = np.linspace(0.0, 0.15, 61)
plt.plot(cx, opt[0] + opt[1] * cx,
         'r')
plt.plot(opt[2], f(opt[2]),
         'y*',
         markersize=15.0)
plt.grid(True)
plt.axhline(0,
            color='k',
            ls='--')
plt.axvline(0,
            color='k',
            ls='--')
plt.xlim([-0.01, 0.15])
plt.ylim([-0.01, 0.2])

plt.xlabel('Volatility of returns')
plt.ylabel('Expected returns')
plt.title('Efficient frontier and the capital market line using 10 stocks')

In [ ]:
#Obtain the weights of the maximum Sharpe ratio portfolio
def min_sharpe(weights):
    #In the class we assumed that rf=0, but here we need to take it into consideration
    #In the notebook on portfolio optimization you have the equation for Sharpe ratio
    return -(portfolio_ret(weights) - rf) / portfolio_vol(weights)

cons = ({'type': 'eq',
         'fun': lambda x: np.sum(x) - 1})

bnds = tuple((0, 1) for x in range(rets.shape[1]))

optimal = sco.minimize(min_sharpe,
                       [1 / rets.shape[1]] * rets.shape[1],
                       method='SLSQP',
                       bounds=bnds,
                       constraints=cons)

print('Maximum Sharpe ratio is {}.'.format(round(-optimal['fun'], 3)))

In [ ]:
#One way to display the amount invested in each stock is through a bar chart
opt_weights = optimal['x']

plt.figure(figsize=(10,6))
plt.grid(True)
plt.bar(np.arange(len(opt_weights)), (opt_weights * 100000).round(2))
plt.xticks(np.arange(len(opt_weights)), rets.columns)

plt.xlabel('Stock ticker')
plt.ylabel('Amount invested ($)')
plt.title('Amount invested in each stock for the maximum Sharpe ratio')

##### 3. You are worried that the highest weighted stock in your portfolio will go down in price soon, however, you want to keep it in your portfolio (at the last day of your sample) for 6 more months due to tax incentives. How can you use options to hedge your exposure to this stock (illustrate graphically)? How much will this hedging cost you (when calculating option price, think about what parameters you need and how to extract them from your data)?

In [ ]:
#Identify the stock with the highest weight
hw_stock = rets.columns[np.argmax(opt_weights)]
print('Stock with the highest weight in the pottfolio is {}.'.format(hw_stock))

In [ ]:
#The last closing price of the MMC stock
S = round(data[hw_stock][-1], 2)
S

In [ ]:
#Chart the payoffs of the put option and the stock
x_values = np.arange(0, 2*S, 0.01)

plt.figure(figsize=(15, 6))
plt.subplot(1,2,1)
plt.grid(True)
plt.plot(x_values, np.maximum(S-x_values, 0),
         'r')
plt.axhline(0,
            color='k',
            ls='--',
            lw=1)
plt.axvline(0,
            color='k',
            ls='--',
            lw=1)
plt.xlabel('Stock price at maturity')
plt.ylabel('Payoff')
plt.title('Put option')

plt.subplot(1,2,2)
plt.grid(True)
plt.plot(x_values, x_values-S,
         'b')
plt.axhline(0,
            color='k',
            ls='--',
            lw=1)
plt.axvline(0,
            color='k',
            ls='--',
            lw=1)
plt.xlabel('Stock price at maturity')
plt.title('Stock')

In [ ]:
#Chart the payoffs of both instruments on one graph and a combined payoff of the two
plt.figure(figsize=(15, 6))
plt.subplot(1,2,1)
plt.grid(True)
plt.plot(x_values, np.maximum(S-x_values, 0),
         'r')
plt.plot(x_values, x_values-S,
         'b')
plt.axhline(0,
            color='k',
            ls='--',
            lw=1)
plt.axvline(0,
            color='k',
            ls='--',
            lw=1)
plt.xlabel('Stock price at maturity')
plt.ylabel('Payoff')
plt.title('Both instruments')

plt.subplot(1,2,2)
plt.grid(True)
plt.plot(x_values, np.maximum(S-x_values, 0) + (x_values-S),
         'g')
plt.axhline(0,
            color='k',
            ls='--',
            lw=1)
plt.axvline(0,
            color='k',
            ls='--',
            lw=1)
plt.xlabel('Stock price at maturity')
plt.title('Combined')

In [ ]:
#Exposure is the amount (to be hedged) invested in MMC stock
exposure = (opt_weights[np.argmax(opt_weights)] * 100000).round(2)
exposure

In [ ]:
#Because I know how long I want to keep my position (for 6 months), I will hedge using a European option
def simulate(M, I):
    #M: number of time intervals for discretization
    #I: number of paths to be simulated
    
    np.random.seed(15)
    simulated = np.random.standard_normal((M + 1, int(I / 2)))
    simulated = np.concatenate((simulated, -simulated), axis=1)
    simulated = (simulated - simulated.mean()) / simulated.std()
    return simulated

def euro_static(K, option='call'):
    #K: strike price of the option

    sim = simulate(0, I)
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * sim)
    if option == 'call':
        hT = np.maximum(ST - K, 0)
    else:
        hT = np.maximum(K - ST, 0)
    P0 = np.exp(-r * T) * np.mean(hT)
    return P0

S0 = exposure
r = rf
#I use 6 periods to calculate standard deviation, because I try to project for the 6 months period as well
sigma = rets[hw_stock][-6:].std() * np.sqrt(12)
T = 0.5
I = 50000

put_price = euro_static(K=S0, option='put')
print('It will cost ${} to hedge exposure of {} stock for 6 months.'.format(round(put_price, 2), hw_stock))

##### 4. Now suppose you decide to hedge your entire portfolio for the next 6 months. Luckily, your friend who works in an investment bank, offered to underwrite you an option on your entire portfolio. What is the fair price for this option (use covariance matrix to estimate variance of the portfolio)?

In [ ]:
#Now I want to hedge the entire portfolio ($100,000) for the 6 months period
S0 = 100000
r = rf
#Covariance matrix
cov = rets[-6:].cov()
sigma = np.sqrt(np.dot(opt_weights.T, np.dot(cov, opt_weights)) * 12)
T = 0.5
I = 50000

put_price = euro_static(K=S0, option='put')
print('It will cost ${} to hedge exposure of the entire portfolio for 6 months.'.format(round(put_price, 2)))